## Day 16: Proboscidea Volcanium

The sensors have led you to the origin of the distress signal: yet another handheld device, just like the one the Elves gave you. However, you don't see any Elves around; instead, the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.

The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!

You need to get the elephants out of here, quickly. Your device estimates that you have 30 minutes before the volcano erupts, so you don't have time to go back out the way you came in.

You scan the cave for other options and discover a network of pipes and pressure-release valves. You aren't sure how such a system got into a volcano, but you don't have time to complain; your device produces a report (your puzzle input) of each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.

There's even a valve in the room you and the elephants are currently standing in labeled AA. You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?

For example, suppose you had the following scan output:

```
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
```

In [1]:
import re
from dataclasses import dataclass
import collections
import pytest
from icecream import ic

In [30]:
@dataclass
class Cave:
    name: str
    flow_rate: int
    leads_to: list
    opened_at: int = 0
    
    def __str__(self):
        if len(self.leads_to) > 1:
            verb = 'tunnel lead to valve'
            subject = self.leads_to[0]
        else:
            verb = 'tunnels leads to valves'
            subject = ', '.join([v for v in self.leads_to])
        return f"Valve {self.name} has flow rate={self.flow_rate}; {verb} {subject}"
    
    @property
    def is_closed(self):
        return self.opened_at == 0
    
    def total_flow(self):
        return self.opened_at * self.flow_rate
    
    def open_valve(self, clock):
        self.closed = False
        self.opened_at = clock
    

In [31]:
pat_valve = re.compile('Valve ([A-Z]+) has flow rate=(\d+); tunnels? leads? to valves? (.+)')

def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            _match = pat_valve.match(line)
            yield Cave(
                name=_match.group(1),
                flow_rate=int(_match.group(2)),
                leads_to=[x for x in _match.group(3).split(', ')],
            )

In [32]:
for c in load_input('sample.txt'):
    print(c)

Valve AA has flow rate=0; tunnel lead to valve DD
Valve BB has flow rate=13; tunnel lead to valve CC
Valve CC has flow rate=2; tunnel lead to valve DD
Valve DD has flow rate=20; tunnel lead to valve CC
Valve EE has flow rate=3; tunnel lead to valve FF
Valve FF has flow rate=0; tunnel lead to valve EE
Valve GG has flow rate=0; tunnel lead to valve FF
Valve HH has flow rate=22; tunnels leads to valves GG
Valve II has flow rate=0; tunnel lead to valve AA
Valve JJ has flow rate=21; tunnels leads to valves II


All of the valves begin closed. You start at valve AA, but it must be damaged or jammed or something: its flow rate is 0, so there's no point in opening it. However, you could spend one minute moving to valve BB and another minute opening it; doing so would release pressure during the remaining 28 minutes at a flow rate of 13, a total eventual pressure release of 28 * 13 = 364. Then, you could spend your third minute moving to valve CC and your fourth minute opening it, providing an additional 26 minutes of eventual pressure release at a flow rate of 2, or 52 total pressure released by valve CC.

In [33]:
cave_b = Cave(name='BB', flow_rate=13, leads_to=['CC'])
assert cave_b.name == 'BB'
assert cave_b.is_closed is True
assert cave_b.flow_rate == 13
assert cave_b.opened_at == 0
assert ic(cave_b.leads_to) == ['CC']

cave_b.open_valve(28)
assert ic(cave_b.total_flow()) == 364

ic| cave_b.leads_to: ['CC']
ic| cave_b.total_flow(): 364


Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed. However, you need to release as much pressure as possible, so you'll need to be methodical. Instead, consider this approach:

```
== Minute 1 ==
No valves are open.
You move to valve DD.

== Minute 2 ==
No valves are open.
You open valve DD.

== Minute 3 ==
Valve DD is open, releasing 20 pressure.
You move to valve CC.

== Minute 4 ==
Valve DD is open, releasing 20 pressure.
You move to valve BB.

== Minute 5 ==
Valve DD is open, releasing 20 pressure.
You open valve BB.

== Minute 6 ==
Valves BB and DD are open, releasing 33 pressure.
You move to valve AA.

== Minute 7 ==
Valves BB and DD are open, releasing 33 pressure.
You move to valve II.

== Minute 8 ==
Valves BB and DD are open, releasing 33 pressure.
You move to valve JJ.

== Minute 9 ==
Valves BB and DD are open, releasing 33 pressure.
You open valve JJ.

== Minute 10 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve II.

== Minute 11 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve AA.

== Minute 12 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve DD.

== Minute 13 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve EE.

== Minute 14 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve FF.

== Minute 15 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve GG.

== Minute 16 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You move to valve HH.

== Minute 17 ==
Valves BB, DD, and JJ are open, releasing 54 pressure.
You open valve HH.

== Minute 18 ==
Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
You move to valve GG.

== Minute 19 ==
Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
You move to valve FF.

== Minute 20 ==
Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
You move to valve EE.

== Minute 21 ==
Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
You open valve EE.

== Minute 22 ==
Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
You move to valve DD.

== Minute 23 ==
Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
You move to valve CC.

== Minute 24 ==
Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
You open valve CC.

== Minute 25 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

== Minute 26 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

== Minute 27 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

== Minute 28 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

== Minute 29 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

== Minute 30 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.
```

This approach lets you release the most pressure possible in 30 minutes with this valve layout, **1651**.

Work out the steps to release the most pressure in 30 minutes. What is the most pressure you can release?

> El primer paso sera obtener el mapa de rutas optimas entre cada una de las cuevas. Si fuera desde cda cueva a una en concreto, podríamos usar 
[Dijkstra](https://es.wikipedia.org/wiki/Algoritmo_de_Dijkstra) o [A*](https://es.wikipedia.org/wiki/Algoritmo_de_b%C3%BAsqueda_A*). Aquí hay [una excelente introducción a A*, pasando antes por Dijkstra](https://www.redblobgames.com/pathfinding/a-star/introduction.html), pero para calcular todas las posibles rutas, un buen algoritmo a usar podría ser el [algorítomo de Floyd-Warshall](https://es.wikipedia.org/wiki/Algoritmo_de_Floyd-Warshall), porque calcula las distancias mínimas entre todas las combinaciones posibles de nodos.

In [ ]:
INFINITE = float('inf')

def load_data(filename):
    distances = collections.defaultdict(lambda: INFINITE)
    caves = {
        cave.name: cave
        for cave in load_input(filename)
    }
    names = list(caves.keys())
    for name in names:
        cave = caves[name]
        distances[name, name] = 0
        for neighbour in cave.leads_to:
            distances[name, neighbour] = 1
    return caves, distances

       
def print_distances(distances):
    names = list(caves.keys())
    print('        ', *[f'{n:>8}' for n in names], sep='')
    for name_from in names:
        print(f'{name_from:>8}', end='')
        for name_to in names:
            d = distances[name_from, name_to]
            if d == INFINITE:
                print('        ', end='')
            else:
                print(f'{d:8}', end='')
        print()
        

caves, distances = load_data('sample.txt')
print_distances(distances)

In [ ]:
def floyd_warshall(caves, distances):
    names = list(caves.keys())
    for r in names:
        for p in names:
            for q in names:
                if r != p and r != q and p != q:
                    distances[p, q] = min(distances[p, q], distances[p, r] + distances[r, q])
    return distances

caves, distances = load_data('sample.txt')
distances = floyd_warshall(caves, distances)
print_distances(distances)

In [ ]:
caves, distances = load_data('sample.txt')
distances = floyd_warshall(caves, distances)
assert distances['AA', 'HH'] == 5
assert distances['JJ', 'II'] == distances['II', 'JJ']== 1
assert distances['GG', 'II'] == distances['II', 'GG']== 5

Estando en un momento determinado en una parte del nodo, tenemos que poder conseguir un listado de todas
las opciones que tenemos:
    
- Si el nodo tiene un caudal positivo, y la valvula está cerrada, abrir la váłvula

- Si queda alguna válvula por abrir, para cada nodo adyacente, ir a ese nodo
    
Las dos operaciones tendrían el mismo coste o efecto: Avanzar el reloj un minuto

Cuando no quede ninguna cueva con caudal positivo y válvula cerrada, ya solo quedaría calcular el flujo total.

In [ ]:
start_at = 'AA'
caves, distances = load_data('sample.txt')
distances = floyd_warshall(caves, distances)

to_open = sum([c.closed and c.flow_rate > 0 for c in caves.values()])
assert to_open == 6

class Command:
    OPEN = 1
    MOVE = 2
    
    def __init__(self, op, target):
        self.op = op
        self.target = target
        
    def __str__(self):
        if self.op == Command.OPEN:
            return f'open valve in cave {self.target.name}'
        elif self.op == Command.MOVE:
            return f'move to cave {self.target.name}'
        
    __repr__ = __str__
    
    @classmethod
    def move(cls, target):
        return cls(cls.MOVE, target)
        
    @classmethod
    def open(cls, target):
        return cls(cls.OPEN, target)

    
def get_options(caves, location):
    cave = caves[location]
    if cave.closed and cave.flow_rate > 0:
        yield Command.open(cave)
    for neighbour in cave.leads_to:
        cave_to = caves[neighbour]
        yield Command.move(cave_to)
        
    
list(get_options(caves, 'DD'))

In [ ]:
def solution_one(filename)
    caves, initial_distances = load_data(filename)    
    clock = 30
    start_at = 'AA'
    location = caves[start_at]
    distances = floyd_warshall(self.caves, initial_distances)
    valves_to_open = sum([c.closed and c.flow_rate > 0 for c in caves.values()])
    
    def calculate(caves, location, valves_to_open, clock):
        if valves_to_open == 0:
        # no more viable options, we can further calculate total flow
            acc = 0
            for c in caves:
                acc += c.flow_rate * c.opened_at
            return acc
    else:
        for option in get_options(self.caves, self.location):
        


    for option in get_options(self.caves, self.location):
        state = 
    if node.valves_to_open == 0:
        # no more viable options, we can further calculate total flow
        for c in self.caves:
            acc += c.flow_rate * self.opened_at
        return acc
    else:
        for option in get_options(self.caves, self.location):
        
    

def solution_one(filename):
    game = Game(filename)
    
    
